## Imports

In [ ]:
import sys

print("python：", sys.version)

import gc

import pkg_resources

python： 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


/tmp/ipykernel_9993/1559013524.py:7: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [ ]:
def get_package_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        return version
    except pkg_resources.DistributionNotFound:
        return "Package not found"

In [ ]:
package_name = "torch"
version = get_package_version(package_name)
print(f"{package_name}：{version}")

torch：2.3.0


In [ ]:
package_name = "transformers"
version = get_package_version(package_name)
print(f"{package_name}：{version}")

transformers：4.40.1


In [ ]:
package_name = "pandas"
version = get_package_version(package_name)
print(f"{package_name}：{version}")

pandas：2.2.2


In [ ]:
import math
import random
import re
import subprocess
import sys
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from numpy.random import choice
from tqdm import tqdm

In [ ]:
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    set_seed,
)

In [ ]:
import torch

cuda_version = torch.version.cuda
print("CUDA：", cuda_version)

CUDA： 12.1


In [ ]:
import time

In [ ]:
## Config
class Config:
    SEED = 42  # SEEDING
    DEBUG = False
    QUANT = True  # To use quantised version or not
    PRIVATE = False  # True for Kaggle submission
    USE_PAST_KEY = True
    MEM_EFFICIENT_SDP = False  # enable memory efficient scaled dot product
    n_repetitions = 19 if PRIVATE else 3  # Number of answers to generate
    TOTAL_TOKENS = 2048
    # TIME_LIMIT = 31500 if PRIVATE else 1
    TIME_LIMIT = 31500

    #     MODEL_PATH = "/kaggle/input/deepseek-math"
    # MODEL_PATH =  "/kaggle/input/open-math-mistral"
    MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"

    ENABLE_GRADIENT_CHECKPOINTING = True

    temperature = 0.9
    top_p = 1.0

    temperature_coding = 0.9
    top_p_coding = 1.0

In [ ]:
cnfg = Config()

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(cnfg.MEM_EFFICIENT_SDP)

In [ ]:
%%time
if cnfg.QUANT:
    from transformers import BitsAndBytesConfig

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    # !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    # !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq


set_seed(cnfg.SEED)

CPU times: user 4.26 ms, sys: 181 µs, total: 4.44 ms
Wall time: 4.22 ms


## Utils

In [ ]:
def naive_parse(answer):
    """
    Takes the generated text and parses the numerical text in the end.
    """
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in "0123456789" and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True

    out = reversed(out)
    return "".join(out)

In [ ]:
def return_last_print(output, n):
    """
    Return the nth line from the given output
    Used for getting the last print from the generated code.
    """
    lines = output.strip().split("\n")
    if lines:
        return lines[n]
    else:
        return ""

In [ ]:
def process_code(code, return_shell_output=False):
    """
    Given the generated code this functions executes and returns the output
    """

    def repl(match):
        if "real" not in match.group():
            return "{}{}".format(match.group()[:-1], ", real=True)")
        else:
            return "{}{}".format(match.group()[:-1], ")")

    code = re.sub(r"symbols\([^)]+\)", repl, code)

    if return_shell_output:
        code = code.replace("\n", "\n    ")
        # Add a try...except block
        code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(
            code
        )

    if not return_shell_output:
        print(code)
    with open("code.py", "w") as fout:
        fout.write(code)

    batcmd = "timeout 7 " + sys.executable + " code.py"
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode("utf8")
        return_value = return_last_print(shell_output, -1)
        print(shell_output)
        if return_shell_output:
            if return_value == "FAIL":
                CODE_STATUS = False
                return_value = return_last_print(shell_output, -2)
                if "not defined" in return_value:
                    return_value += "\nTry checking the formatting and imports"
            else:
                CODE_STATUS = True
            return return_value, CODE_STATUS
        code_output = round(float(eval(return_value))) % 1000
    except Exception as e:
        print(e, "shell_output")
        code_output = -1

    if return_shell_output:
        if code_output == -1:
            CODE_STATUS = False
        else:
            CODE_STATUS = True
        return code_output, CODE_STATUS

    return code_output

In [ ]:
def process_text_output(output):
    """
    Parsing text output.
    """
    result = output
    try:
        result_output = re.findall(r"\\boxed\{(\d+)\}", result)

        print("BOXED", result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print("BOXED FINAL", result_output)
        if not len(result_output):
            result_output = -1

        else:
            result_output = round(float(eval(result_output))) % 1000

    except Exception as e:
        print(e)
        print("ERROR PARSING TEXT")
        result_output = -1

    return result_output

## Model

In [ ]:
torch.cuda.empty_cache()
gc.collect()

40

In [ ]:
%%time
config = AutoConfig.from_pretrained(cnfg.MODEL_PATH)
config.gradient_checkpointing = cnfg.ENABLE_GRADIENT_CHECKPOINTING

tokenizer = AutoTokenizer.from_pretrained(cnfg.MODEL_PATH)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 197 ms, sys: 48.7 ms, total: 246 ms
Wall time: 589 ms


In [ ]:
# ## For 2 T4 GPUs
# device_map = [('model.embed_tokens', 0),
#              ('model.layers.0', 0),
#              ('model.layers.1', 0),
#              ('model.layers.2', 0),
#              ('model.layers.3', 0),
#              ('model.layers.4', 0),
#              ('model.layers.5', 0),
#              ('model.layers.6', 0),
#              ('model.layers.7', 0),
#              ('model.layers.8', 0),
#              ('model.layers.9', 0),
#              ('model.layers.10', 0),
#              ('model.layers.11', 0),
#              ('model.layers.12', 0),
#              ('model.layers.13', 0),
#              ('model.layers.14', 0),
#              ('model.layers.15', 0),
#              ('model.layers.16', 0),
#              ('model.layers.17', 0),
#              ('model.layers.18', 0),
#              ('model.layers.19', 0),
#              ('model.layers.20', 0),
#              ('model.layers.21', 0),
#              ('model.layers.22', 1),
#              ('model.layers.23', 1),
#              ('model.layers.24', 1),
#              ('model.layers.25', 1),
#              ('model.layers.26', 1),
#              ('model.layers.27', 1),
#              ('model.layers.28', 1),
#              ('model.layers.29', 1),
#              ('model.norm', 1),
#              ('lm_head', 1)]

# if 'open-math' in cnfg.MODEL_PATH:
#     device_map = 'auto'
#     print(device_map)
# else:
#     device_map = {ii:jj for (ii,jj) in device_map}

In [ ]:
device_map = "auto"

In [ ]:
if cnfg.QUANT:
    from transformers import BitsAndBytesConfig

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        cnfg.MODEL_PATH,
        device_map="sequential",
        torch_dtype="auto",
        trust_remote_code=True,
        quantization_config=quantization_config,
        config=config,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        cnfg.MODEL_PATH,
        device_map=device_map,
        torch_dtype="auto",
        trust_remote_code=True,
        # quantization_config=quantization_config,
        config=config,
    )

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype="auto",
    device_map=device_map,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model.dtype, model.hf_device_map

(torch.bfloat16, {'': 0})

In [ ]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops=[], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            last_token = input_ids[0][-len(stop) :]
            if torch.all(torch.eq(stop, last_token)):
                return True
        return False


stop_words = ["```output", "```python", "```\nOutput", ")\n```", "``````output"]  # ,
stop_words_ids = [
    tokenizer(stop_word, return_tensors="pt", add_special_tokens=False)[
        "input_ids"
    ].squeeze()
    for stop_word in stop_words
]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

## Prompts

In [ ]:
# Chain of thought
cot1 = """Here are few sample math problems and there solution with explanation without program:

1) Sample Problem 1:

\"A bag has 3 red marbles and 5 white marbles.  Two marbles are drawn from the bag and not replaced.  What is the probability that the first marble is red and the second marble is white?\"

Solution for sample problem 1:

\"The probability that the first is red is $\dfrac38$.  Now with 7 remaining, the probability that the second is white is $\dfrac57$. The answer is $\dfrac38 \times \dfrac57 = \boxed{\dfrac{15}{56}}$.\"


2) Sample Problem 2:

\"A certain integer has $4$ digits when written in base $8$. The same integer has $d$ digits when written in base $2$. What is the sum of all possible values of $d$?\"

Solution for sample problem 2:

\"The smallest integer that has $4$ digits in base $8$ is $1000_8$, which stands for $8^3 = 2^9$. The largest integer that has $4$ digits in base $8$ is $7777_8$, which is $1$ less than $10000_8$ and therefore stands for $8^4-1 = 2^{12}-1$.

Thus, when a $4$-digit base-$8$ integer is written in base $2$, its highest place value is either $2^9$, $2^{10}$, or $2^{11}$. It follows that the base-$2$ expression has $10$, $11$, or $12$ digits, so the sum of all possible values for $d$ is $10+11+12 = \boxed{33}$.\"



Now below is a math problem you are to solve (positive numerical answer!):
\"{..rohit..}\"

Analyze this problem, give an explanation and then come to a solution with programs. 

After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

In [ ]:
# Chain of thought
cot2 = """Here are few sample math problems and there solution with explanation without program:

1) Sample Problem 1:

\"A bag has 3 red marbles and 5 white marbles.  Two marbles are drawn from the bag and not replaced.  What is the probability that the first marble is red and the second marble is white?\"

Solution for sample problem 1:

\"The probability that the first is red is $\dfrac38$.  Now with 7 remaining, the probability that the second is white is $\dfrac57$. The answer is $\dfrac38 \times \dfrac57 = \boxed{\dfrac{15}{56}}$.\"


2) Sample Problem 2:

\"A certain integer has $4$ digits when written in base $8$. The same integer has $d$ digits when written in base $2$. What is the sum of all possible values of $d$?\"

Solution for sample problem 2:

\"The smallest integer that has $4$ digits in base $8$ is $1000_8$, which stands for $8^3 = 2^9$. The largest integer that has $4$ digits in base $8$ is $7777_8$, which is $1$ less than $10000_8$ and therefore stands for $8^4-1 = 2^{12}-1$.

Thus, when a $4$-digit base-$8$ integer is written in base $2$, its highest place value is either $2^9$, $2^{10}$, or $2^{11}$. It follows that the base-$2$ expression has $10$, $11$, or $12$ digits, so the sum of all possible values for $d$ is $10+11+12 = \boxed{33}$.\"



Now below is a math problem you are to solve (positive numerical answer!):
\"{..rohit..}\"

Analyze this problem, give an explanation and then come to a solution without programs. 

After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

In [ ]:
tool_instruction = "\n\nPlease integrate natural language reasoning with programs to solve the above problem, and put your final numerical answer within \\boxed{}.\nNote that the intermediary calculations may be real numbers, but the final numerical answer would always be an integer."

In [ ]:
promplt_options = [cot1, cot2]

### Read Data

In [ ]:
data_df = pd.read_csv("./data/aimo_given/train.csv")

In [ ]:
data_df.head()

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211


In [ ]:
sample_df = data_df.sample(2)

In [ ]:
sample_df

,id,problem,answer
5,739bc9,For how many positive integers $m$ does the eq...,199
4,5277ed,There exists a unique increasing geometric seq...,211


In [ ]:
sample_df.iloc[0].problem

'For how many positive integers $m$ does the equation \\[\\vert \\vert x-1 \\vert -2 \\vert=\\frac{m}{100}\\] have $4$ distinct solutions?'

In [ ]:
cot.replace("{..rohit..}", sample_df.iloc[0].problem)

'Here are few sample math problems and there solution with explanation without program:\n\n1) Sample Problem 1:\n\n"A bag has 3 red marbles and 5 white marbles.  Two marbles are drawn from the bag and not replaced.  What is the probability that the first marble is red and the second marble is white?"\n\nSolution for sample problem 1:\n\n"The probability that the first is red is $\\dfrac38$.  Now with 7 remaining, the probability that the second is white is $\\dfrac57$. The answer is $\\dfrac38 \times \\dfrac57 = \x08oxed{\\dfrac{15}{56}}$."\n\n\n2) Sample Problem 2:\n\n"A certain integer has $4$ digits when written in base $8$. The same integer has $d$ digits when written in base $2$. What is the sum of all possible values of $d$?"\n\nSolution for sample problem 2:\n\n"The smallest integer that has $4$ digits in base $8$ is $1000_8$, which stands for $8^3 = 2^9$. The largest integer that has $4$ digits in base $8$ is $7777_8$, which is $1$ less than $10000_8$ and therefore stands for $

## Inference

In [ ]:
NOTEBOOK_START_TIME = time.time()

In [ ]:
total_results = {}
total_answers = {}
best_stats = {}
total_outputs = {}
question_type_counts = {}
starting_counts = (2, 3)  # to give probability to promts

# for i, (test, sample_submission) in tqdm(enumerate(iter_test)):
for i, (_, test) in tqdm(enumerate(sample_df.iterrows())):
    print(f"Solving problem {i} ...")
    TIME_SPENT = time.time() - NOTEBOOK_START_TIME

    if TIME_SPENT > cnfg.TIME_LIMIT:
        print(TIME_SPENT, cnfg.TIME_LIMIT)
        break

    for jj in tqdm(range(cnfg.n_repetitions)):
        # problem = test["problem"].values[0]
        problem = test["problem"]
        print(f"\n\n\nQUESTION {i} - {jj} - TIME_SPENT : {TIME_SPENT:.0f} secs")

        best, best_count = best_stats.get(i, (-1, -1))
        if best_count > np.sqrt(jj):
            print("SKIPPING CAUSE ALREADY FOUND BEST")
            continue

        outputs = total_outputs.get(i, [])
        text_answers, code_answers = question_type_counts.get(i, starting_counts)
        results = total_results.get(i, [])
        answers = total_answers.get(i, [])

        for _ in range(5):
            torch.cuda.empty_cache()
            gc.collect()
            time.sleep(0.2)

        try:
            ALREADY_GEN = 0
            code_error = None
            code_error_count = 0
            code_output = -1
            # initail_message = problem  + tool_instruction
            counts = np.array([text_answers, code_answers])

            draw = choice(promplt_options, 1, p=counts / counts.sum())  # cot has mo

            # initail_message = draw[0].format(problem, "{}")
            initail_message = draw[0].replace("{..rohit..}", problem)
            prompt = f"User: {initail_message}"

            current_printed = len(prompt)
            print(f"{jj}_{prompt}\n")

            model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
            input_len = len(model_inputs["input_ids"][0])

            generation_output = model.generate(
                **model_inputs,
                max_new_tokens=cnfg.TOTAL_TOKENS - ALREADY_GEN,
                return_dict_in_generate=cnfg.USE_PAST_KEY,
                do_sample=True,
                temperature=cnfg.temperature,
                top_p=cnfg.top_p,
                num_return_sequences=1,
                stopping_criteria=stopping_criteria,
            )

            if cnfg.USE_PAST_KEY:
                output_ids = generation_output.sequences[0]
            else:
                output_ids = generation_output[0]
            decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
            print(f"{decoded_output[current_printed:]}\n")
            current_printed += len(
                decoded_output[current_printed:]
            )  # Size of prompt + Size of generated text

            cummulative_code = ""

            stop_word_cond = False  # Whether last word generated is a stop word or not.
            for stop_word in stop_words:
                stop_word_cond = stop_word_cond or (
                    decoded_output[-len(stop_word) :] == stop_word
                )
                if cnfg.DEBUG:
                    if stop_word_cond:
                        print(f"***** 1: Stop Word: {stop_word}")

            # while stop word is generated and not generated maximum tokens (cnfg.TOTAL_TOKENS)
            while (stop_word_cond) and (ALREADY_GEN < (cnfg.TOTAL_TOKENS)):
                if cnfg.DEBUG:
                    print(f"****** 2 ALREADY_GEN:{ALREADY_GEN}, {cnfg.TOTAL_TOKENS}")

                if decoded_output[-len("```python") :] == "```python":
                    if cnfg.DEBUG:
                        print(f"****** 3 Stop Word: ```python found")
                    temperature_inner = cnfg.temperature_coding
                    top_p_inner = cnfg.top_p_coding
                    prompt = decoded_output
                else:
                    if cnfg.DEBUG:
                        print(f"****** 4 looking for other stop word")
                    temperature_inner = cnfg.temperature
                    top_p_inner = cnfg.top_p
                    try:
                        if decoded_output[-len("``````output") :] == "``````output":
                            if cnfg.DEBUG:
                                print(f"****** 5 Stop word ``````output found")
                            code_text = decoded_output.split("```python")[-1].split(
                                "``````"
                            )[0]
                        else:
                            code_text = decoded_output.split("```python")[-1].split(
                                "```"
                            )[0]

                        cummulative_code += code_text

                        if cnfg.DEBUG:
                            print(
                                f"****** 6 Found Code ************ \n {cummulative_code} \n ***********************"
                            )
                        code_output, CODE_STATUS = process_code(
                            cummulative_code, return_shell_output=True
                        )
                        print("CODE RESULTS", code_output)

                        if code_error == code_output:
                            code_error_count += 1
                        else:
                            code_error = code_output
                            code_error_count = 0

                        if not CODE_STATUS:
                            cummulative_code = cummulative_code[: -len(code_text)]
                            if cnfg.DEBUG:
                                print(
                                    f"****** 7 CODE STATUS {CODE_STATUS} New  cummulative_code ************ \n {cummulative_code} \n ***********************"
                                )

                            if code_error_count >= 1:
                                print("REPEATED ERRORS")
                                break

                    except Exception as e:
                        print(e)
                        print("ERROR PARSING CODE")
                        code_output = -1

                    if code_output != -1:
                        if decoded_output[-len(")\n```") :] == ")\n```":
                            prompt = (
                                decoded_output
                                + "```output\n"
                                + str(code_output)
                                + "\n```\n"
                            )
                            if cnfg.DEBUG:
                                print(
                                    f"****** 8 New prompt with code output ************ \n {prompt} \n ***********************"
                                )
                        else:
                            prompt = (
                                decoded_output + "\n" + str(code_output) + "\n```\n"
                            )
                            if cnfg.DEBUG:
                                print(
                                    f"****** 9 New prompt with code output ************ \n {prompt} \n ***********************"
                                )
                    else:
                        prompt = decoded_output
                        cummulative_code = ""
                        if cnfg.DEBUG:
                            print(
                                f"****** 10 Code out put -1 new prompt ************ \n {prompt} \n ***********************"
                            )

                model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
                ALREADY_GEN = len(model_inputs["input_ids"][0]) - input_len

                if cnfg.USE_PAST_KEY:
                    old_values = generation_output.past_key_values
                else:
                    old_values = None

                generation_output = model.generate(
                    **model_inputs,
                    max_new_tokens=cnfg.TOTAL_TOKENS - ALREADY_GEN,
                    return_dict_in_generate=cnfg.USE_PAST_KEY,
                    past_key_values=old_values,
                    do_sample=True,
                    temperature=temperature_inner,
                    top_p=top_p_inner,
                    num_return_sequences=1,
                    stopping_criteria=stopping_criteria,
                )

                if cnfg.USE_PAST_KEY:
                    output_ids = generation_output.sequences[0]
                else:
                    output_ids = generation_output[0]
                decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
                print(f"\nINTERMEDIATE OUT :\n{decoded_output[current_printed:]}\n")
                current_printed += len(decoded_output[current_printed:])

                stop_word_cond = False
                for stop_word in stop_words:
                    stop_word_cond = stop_word_cond or (
                        decoded_output[-len(stop_word) :] == stop_word
                    )

            if cnfg.USE_PAST_KEY:
                output_ids = generation_output.sequences[0]
            else:
                output_ids = generation_output[0]

            raw_output = tokenizer.decode(
                output_ids[input_len:], skip_special_tokens=True
            )
            # print(f"\n\nOutput :\n{raw_output}\n")
            result_output = process_text_output(raw_output)

            try:
                code_output = round(float(eval(code_output))) % 1000
            except Exception as e:
                print(e, "final_eval")
                code_output = -1

        except Exception as e:
            print(e, "5")
            result_output, code_output = -1, -1

        if code_output != -1:
            outputs.append(code_output)
            code_answers += 1

        if result_output != -1:
            outputs.append(result_output)
            text_answers += 1

        if len(outputs) > 0:
            occurances = Counter(outputs).most_common()
            print(occurances)
            if occurances[0][1] > best_count:
                print("GOOD ANSWER UPDATED!")
                best = occurances[0][0]
                best_count = occurances[0][1]
            if occurances[0][1] > 5:
                print("ANSWER FOUND!")
                break

        results.append(result_output)
        answers.append(code_output)

        best_stats[i] = (best, best_count)
        question_type_counts[i] = (text_answers, code_answers)
        total_outputs[i] = outputs

        total_results[i] = results
        total_answers[i] = answers

        print(
            "code_answers",
            code_answers - starting_counts[1],
            "text_answers",
            text_answers - starting_counts[0],
        )
        if cnfg.DEBUG:
            break

    print(f"Predicted best answer: {best_stats}")

0it [00:00, ?it/s]

Solving problem 0 ...



  0%|                                                     | 0/3 [00:00<?, ?it/s]




QUESTION 0 - 0 - TIME_SPENT : 0 secs


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


...


INTERMEDIATE OUT :

def sum_of_sequence():
    """There exists a unique increasing geometric sequence of five 2-digit positive integers. What is their sum?"""
    for a in range(10, 100):
        for r in range(2, 10):
            sequence = [a * r**i for i in range(5)]
            if all(10 <= n < 100 for n in sequence) and sorted(sequence) == sequence:
                sum_of_sequence = sum(sequence)
                return sum_of_sequence

result = sum_of_sequence()
print(result)
``````output

None
None
None
None
None
None
None
None

CODE RESULTS None


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.



INTERMEDIATE OUT :

None
```
The output is still `None` because the script is still not properly handling the condition that the sequence should be increasing. Let's modify the script to handle the condition properly and return the sum of the sequence when it's found.

```python




100%|█████████████████████████████████████████████| 3/3 [03:21<00:00, 67.20s/it]
2it [04:11, 125.79s/it]


INTERMEDIATE OUT :

def sum_of_sequence():
    """There exists a unique increasing geometric sequence of five 2-digit positive integers. What is their sum?"""
    for a in range(10, 100):
        for r in range(2, 10):
            sequence = [a * r**i for i in range(5)]

BOXED []
BOXED FINAL 5
float() argument must be a string or a real number, not 'NoneType' final_eval
[(166, 1), (31, 1), (5, 1)]
code_answers 0 text_answers 3
Predicted best answer: {0: (100, 1), 1: (166, 1)}


In [ ]:
best_stats

{0: (100, 1), 1: (166, 1)}

In [ ]:
sample_df

,id,problem,answer
5,739bc9,For how many positive integers $m$ does the eq...,199
4,5277ed,There exists a unique increasing geometric seq...,211
